In [43]:
# Read configuration file
import configparser
import os
import sys
import time
import logging

# Configure logging
logging.basicConfig(filename='aprioriFC.log',
                    level=logging.ERROR,
                    format='%(asctime)s %(levelname)s:%(message)s')

def log_error(message):
    logging.error(message)

config_path = r'C:\Users\PC\Dropbox\Projects\github\Python\Python\Data Analysis\Store\config\FCconfig.ini'
if not os.path.exists(config_path):
    print('Configuration file not found. Exiting...')
    time.sleep(5)
    sys.exit()

config = configparser.ConfigParser()
config.read(config_path)

# Check if FC data is available
# Read data path from config file
fc_data_path = config['data']['fc_data_folder']

# Configure logging
logging.basicConfig(filename= config['logging']['log_file'],
                    level=logging.ERROR,
                    format='%(asctime)s %(levelname)s:%(message)s')


In [44]:
# Read filter value from config file
filter_value = config.getint('filter', 'filter_value')

# Read data path from config file
fc_data_path = config['data']['fc_data_folder']

# Read apriori parameters from config file
min_support = config.getfloat('apriori', 'min_support')
metric = config.get('apriori', 'metric')

# Read logging parameters from config file
log_file = config['logging']['log_file']

filter_value

1

In [45]:
# Load data detail_trans in fc_data_path FC.db
import pandas as pd
import sqlite3

# Connect to the database
conn = sqlite3.connect(fc_data_path)

# Read data from the database
query = 'SELECT * FROM detail_trans'
df = pd.read_sql(query, conn)

# Close the connection
conn.close()

df.head()

,invoiceDate,invoiceCode,storeCode,employeeCode,employeeName,storeName,barcode,itemName,qty,unitPrice,...,VND,VNPAY,OCB-eBank,GrabOrder,ShopeeFood,ZaloPay,Membership,e-Voucher,ShopeePay,SonKim Card
0,15/06/2024 06:50:07,VN0265011506240037,VN0265,F231230003,Hồ Tuyết Nhi,GS25 Hoang Anh Thanh Binh_FC,8935049510031,Nutri SCU Hy Lạp hương việt quốc 170ml x Chai,1,11000,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,15/06/2024 18:03:06,VN0265021506240132,VN0265,F240427001,Lê Thị Ngọc Ánh,GS25 Hoang Anh Thanh Binh_FC,8801767336706,NOKCHAWON Trà Đào 170ml x gói,1,26000,...,52000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,15/06/2024 18:03:06,VN0265021506240132,VN0265,F240427001,Lê Thị Ngọc Ánh,GS25 Hoang Anh Thanh Binh_FC,8934743032917,VINAMIT Chuối sấy 100g x Gói,1,26000,...,52000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,15/06/2024 23:59:58,VN0265021506240172,VN0265,F231230009,Tằng Cắm Út,GS25 Hoang Anh Thanh Binh_FC,8935058913366,CP Xúc xích tiệt trùng phô mai 60g x cây,2,14000,...,41000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,15/06/2024 23:59:58,VN0265021506240172,VN0265,F231230009,Tằng Cắm Út,GS25 Hoang Anh Thanh Binh_FC,8934803043730,PINATTSU Đậu phộng vị nước cốt dừa 85g x gói,1,13000,...,41000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [46]:
import sqlite3

# Connect to the database
conn = sqlite3.connect(fc_data_path)

# Load item_category table to df_item_category
query = 'SELECT barcode, categoryName, subCategoryName FROM item_category'
df_item_category = pd.read_sql(query, conn)

# Close the connection
conn.close()

df_item_category.head()


,barcode,categoryName,subCategoryName
0,2010101000005,FF Beverage Onsite,Cup Of Ice
1,2501031190411,FF Beverage Onsite,Cup Of Ice
2,2501031190510,FF Beverage Onsite,Cold Local Drinks
3,8934588873553,Tea,Sugar Tea
4,8934588063053,Bottled Water,Purified water


In [47]:
# Convert barcode columns to string
df['barcode'] = df['barcode'].astype(str)
df_item_category['barcode'] = df_item_category['barcode'].astype(str)

# Merge df and df_item_category
df_merged = pd.merge(df, df_item_category, on='barcode', how='left')

df_merged.head()

,invoiceDate,invoiceCode,storeCode,employeeCode,employeeName,storeName,barcode,itemName,qty,unitPrice,...,OCB-eBank,GrabOrder,ShopeeFood,ZaloPay,Membership,e-Voucher,ShopeePay,SonKim Card,categoryName,subCategoryName
0,15/06/2024 06:50:07,VN0265011506240037,VN0265,F231230003,Hồ Tuyết Nhi,GS25 Hoang Anh Thanh Binh_FC,8935049510031,Nutri SCU Hy Lạp hương việt quốc 170ml x Chai,1,11000,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,Flavored Milk,Flavored Milk Sub
1,15/06/2024 06:50:07,VN0265011506240037,VN0265,F231230003,Hồ Tuyết Nhi,GS25 Hoang Anh Thanh Binh_FC,8935049510031,Nutri SCU Hy Lạp hương việt quốc 170ml x Chai,1,11000,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,Flavored Milk,Flavored Milk Sub
2,15/06/2024 06:50:07,VN0265011506240037,VN0265,F231230003,Hồ Tuyết Nhi,GS25 Hoang Anh Thanh Binh_FC,8935049510031,Nutri SCU Hy Lạp hương việt quốc 170ml x Chai,1,11000,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,Yogurt,Drink
3,15/06/2024 18:03:06,VN0265021506240132,VN0265,F240427001,Lê Thị Ngọc Ánh,GS25 Hoang Anh Thanh Binh_FC,8801767336706,NOKCHAWON Trà Đào 170ml x gói,1,26000,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,Juice,OSC Juice
4,15/06/2024 18:03:06,VN0265021506240132,VN0265,F240427001,Lê Thị Ngọc Ánh,GS25 Hoang Anh Thanh Binh_FC,8934743032917,VINAMIT Chuối sấy 100g x Gói,1,26000,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,Non-potato Chips,Fruit Snacks


In [48]:
print('Filter value:', filter_value)

if filter_value == 1:
    df_filter = df_merged[['invoiceCode','categoryName']]
    df_filter = df_filter[df_filter.groupby('invoiceCode')['categoryName'].transform('size') > 1]
    dataset_str = df_filter.groupby('invoiceCode').agg({
        'categoryName': lambda x: ','.join(str(i) for i in x)
    }).reset_index()
    
elif filter_value == 2:
    df_filter = df_merged[['invoiceCode','subCategoryName']]
    df_filter = df_filter[df_filter.groupby('invoiceCode')['subCategoryName'].transform('size') > 1]
    dataset_str = df_filter.groupby('invoiceCode').agg({
        'subCategoryName': lambda x: ','.join(str(i) for i in x)
    }).reset_index()
else:
    print('Filter value is not correct')

Filter value: 1


In [49]:
dataset_str

,invoiceCode,categoryName
0,VN0265010107240001,"Whole Milk,Whole Milk,Whole Milk"
1,VN0265010107240003,"Coffee Drinks,Coffee Drinks"
2,VN0265010107240005,"Rice Offsite,Rice Offsite,Ham&sausage,Ham&sa..."
3,VN0265010107240006,"Packaged Ice Cream/Novelties,Packaged Ice Crea..."
4,VN0265010107240007,"School Supplies,School Supplies,School Supplies"
...,...,...
54123,VN0265023110240165,"Fruit Cat,Fruit Cat"
54124,VN0265023110240166,"Feminine Hygiene,School Supplies,Feminine Hygi..."
54125,VN0265023110240167,"Showcase bakery,Flavored Milk,Whole Milk"
54126,VN0265023110240168,"Other Cuisine,Cuisine Cat,Potato Chips,Instant..."


In [50]:
def apriopri(df, filter_value, min_support, metric):
    from mlxtend.preprocessing import TransactionEncoder
    from mlxtend.frequent_patterns import apriori, association_rules
    if filter_value == 1:
        data = df[['categoryName']]
    elif filter_value == 2:
        data = df[['subCategoryName']]
    # create a TransactionEncoder object
    data = data.values.tolist()

    data = [x[0].split(',') for x in data]

    te = TransactionEncoder()

    # transform the data into a list of lists
    transactions = te.fit(data).transform(data)
    transactions = pd.DataFrame(transactions, columns=te.columns_)

    # Calculate frequent itemsets using Apriori algorithm
    frequent_itemsets = apriori(transactions, min_support=min_support, use_colnames=True)

    # Calculate rules
    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=0)
    
    # Add a new column to the rules DataFrame to calculate the conviction
    rules['conviction'] = (1 - rules['support']) / (1 - rules['confidence'])
    return rules


In [51]:
# Get StoreID from InvoiceID
dataset_str['StoreID'] = dataset_str['invoiceCode'].map(lambda x: x[:6])
# Convert the date string to a datetime object and extract the month
dataset_str['Month'] = dataset_str['invoiceCode'].str[10:12].map(lambda x: int(x[:2]))


In [52]:
# List of unique months
months = dataset_str['Month'].unique()

In [53]:
# List of unique stores
stores = dataset_str['StoreID'].unique()

# Sum number of stores
len(stores)

1

In [54]:
dataset_str.head()

,invoiceCode,categoryName,StoreID,Month
0,VN0265010107240001,"Whole Milk,Whole Milk,Whole Milk",VN0265,7
1,VN0265010107240003,"Coffee Drinks,Coffee Drinks",VN0265,7
2,VN0265010107240005,"Rice Offsite,Rice Offsite,Ham&sausage,Ham&sa...",VN0265,7
3,VN0265010107240006,"Packaged Ice Cream/Novelties,Packaged Ice Crea...",VN0265,7
4,VN0265010107240007,"School Supplies,School Supplies,School Supplies",VN0265,7


In [55]:
# Print unique month in dataset_str
dataset_str['Month'].unique

<bound method Series.unique of 0         7
1         7
2         7
3         7
4         7
         ..
54123    10
54124    10
54125    10
54126    10
54127    10
Name: Month, Length: 54128, dtype: int64>

In [56]:
import os
fail_store = []
apr_results_file = config.get('apriori', 'apr_results_file')
# Clear all files in the folder
for file in os.listdir(apr_results_file):
    file_path = os.path.join(apr_results_file, file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as e:
        print(e)
for i in range(1,len(months)+1):
    month_df = dataset_str[dataset_str['Month'] == i]
    for store in stores:
        store_df = month_df[month_df['StoreID'] == store]
        try:
            rules = apriopri(store_df, filter_value, min_support, metric)
            rules['StoreID'] = store
            rules['Month'] = i
            # change rules columns order
            rules = rules[['StoreID', 'Month', 'antecedents', 'consequents', 'support', 'lift', 'confidence', 'conviction']]
            # Rename columns
            rules.columns = ['StoreID', 'Month', 'Trigger', 'Dependence', 'Support', 'Lift', 'Confidence', 'Conviction']
            if filter_value == 1:
                file_output = os.path.join(apr_results_file, 'Result_M' + str(i) + '_' + 'Cate_Filter' +  str(min_support) + '.csv')
            elif filter_value == 2:
                file_output = os.path.join(apr_results_file, 'Result_M' + str(i) + '_' + 'SubCate_Filter' +  str(min_support) + '.csv')
            else:
                print('Filter value is not correct')
            # Check if the file already exists, if not create a new file
            if not os.path.isfile(file_output):
                with open(file_output, 'w') as f:
                    f.write('')
            with open(file_output, 'r') as f:
                first_line = f.readline().strip()
                # Check if the first line contains the expected column names
                has_headers = 'support' in first_line
            rules.to_csv(file_output, index=False, mode='a', header= not has_headers)
        except Exception as e:
            fail_store.append('Store ' + store + ' Month ' + str(i) + ' ' + str(e))
# Make empty error.log file
open(log_file, 'w').close()

# Write the failed stores to error.log file
with open(log_file, 'w') as f:
    for item in fail_store:
        f.write("%s\n" % item)